Import libraries

In [1]:
import requests
import json
import re
import pandas as pd
from newspaper import Article
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup  
from time import sleep
from selenium import webdriver

# Part I: Scraping Title, Text, and Tags

Create function to minimize clutter

In [2]:
def make_request1(url):
    # MAKE HTTP POST REQUEST  
    page = ''
    while page == '':
        # IN CASE OF TOO MANY REQUESTS
        try:
            page = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urlopen(page)
            if webpage.getcode() == 200:
                break
        except Exception as inst:
            print("Connection refused by the server..")
            print("Let me sleep for 5 seconds")
            print("ZZzzzz...")
            sleep(30)
            print("Was a nice sleep, now let me continue...")
            continue
    return page

Gather links for articles from January 2020 to July 2020

In [3]:
# STORE HTML CODE FOR EACH PAGE
store_pages=[] 

# SET TIME PARAMETERS
months = ['01','02','03','04','05','06','07'] 
days29 = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29'] 
days30 = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30'] 
days31 = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31'] 

for i in months: # LOOP THROUGH MONTHS
    
    if i == '02': # FEBRUARY 2020 HAS 29 DAYS IN A MONTH
        print(i)
        for j in days29: # LOOP THROUGH DAYS
            
            url ='https://towardsdatascience.com/archive/2020/'+ i + '/' + j
            page = make_request1(url)
            
            # PARSE PAGE
            webpage = urlopen(page).read()
            soup = BeautifulSoup(webpage, "html.parser")
            store_pages.append(soup) # STORE EACH HTML PARSED PAGE IN LIST 
            sleep(5)
            
    elif i == '04' or i == '06': # APRIL AND JUNE HAVE 30 DAYS IN A MONTH
        print(i)
        for j in days30: # LOOP THROUGH DAYS
            
            url ='https://towardsdatascience.com/archive/2020/'+ i + '/' + j
            page = make_request1(url)
            
            # PARSE PAGE
            webpage = urlopen(page).read()
            soup = BeautifulSoup(webpage, "html.parser")
            store_pages.append(soup) # STORE EACH HTML PARSED PAGE IN LIST 
            sleep(5)
            
    else: # ALL OTHER MONTHS HAVE 31 DAYS 
        print(i)
        for j in days31: # LOOP THROUGH DAYS
            
            url ='https://towardsdatascience.com/archive/2020/'+ i + '/' + j
            page = make_request1(url)
            
            # PARSE PAGE
            webpage = urlopen(page).read()
            soup = BeautifulSoup(webpage, "html.parser")
            store_pages.append(soup) # STORE EACH HTML PARSED PAGE IN LIST 
            sleep(5)
            
# RETRIEVE LINKS
links_tds= [] 
for i in range(0, len(store_pages)):
    for link in store_pages[i].find_all("a", {"class": "button button--smaller button--chromeless u-baseColor--buttonNormal"}):
        links_tds.append(link.get('href'))
print(len(links_tds),'articles')

01
Connection refused by the server..
Let me sleep for 5 seconds
ZZzzzz...
Was a nice sleep, now let me continue...
02
03
04
05
06
07
11585 articles


Pull relevant data from each article (11,585 total articles)

In [5]:
# STORE RESULTS
article_info_tds=[] 
df_tds=[]

for i in range(0,len(links_tds)): # LOOP THROUGH LINKS

    # TO TRACK PROGRESS
    if (i/1000) % 2 == 1 or (i/1000) % 2 == 0:
        print('article number', i+1)
        
    # MAKE HTTP GET REQUEST
    url = links_tds[i]
    page = ''
    while page == '':
    # IN CASE OF TOO MANY REQUESTS
        try:
            page = requests.get(url)
            if page.status_code == 200:
                break
        except Exception as inst:
            print("Connection refused by the server..")
            print("Let me sleep for 30 seconds")
            print("ZZzzzz...")
            sleep(30)
            print("Was a nice sleep, now let me continue...")
            continue
           
    # PARSE PAGE
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # TITLE (REMEMBER, SOME TITLES MAY NOT HAVE h1 TAG)
    title = []
    for my_tag in soup.find_all('h1'):
        title.append(my_tag.text)
    if title != []:
        title = title[0]
    else:
        title = ''
        
#     # FIGURES
#     figs = []
#     for my_tag in soup.find_all('figure'):
#         if my_tag != []:
#             figs.append(my_tag['class'])
#         else:
#             figs = ''
    
    # TEXT
    body = []
    for my_tag in soup.find_all('p'):
        body.append(my_tag.text)
    
    # TAGS
    tags = []
    for my_tag in soup.find_all('a', href=True):
        if my_tag['href'].startswith('/tagged'): # ALL TAGS HAVE '/tagged' PREFIX
            tags.append(my_tag['href'])
    
    # LINK
    link = url
     
    # ACCUMULATE DATA INTO A DICTIONARY 
    article_info_tds = {'Title':title,'Text':body,'Tags':tags,'Url':link} 

    # STORE RESULTS IN LIST
    df_tds.append(article_info_tds)
    sleep(5)

# CONVERT LIST TO DATAFRAME
df1 = pd.DataFrame(df_tds) 
df1.head()

article number 1
article number 1001
article number 2001
article number 3001
article number 4001
article number 5001
Connection refused by the server..
Let me sleep for 30 seconds
ZZzzzz...
Was a nice sleep, now let me continue...
article number 6001
article number 7001
article number 8001
article number 9001
article number 10001
article number 11001


,Tags,Text,Title,Url
0,"[/tagged/programming, /tagged/python, /tagged/...","[Python haters always say, that one of the rea...",Making Python Programs Blazingly Fast,https://towardsdatascience.com/making-python-p...
1,"[/tagged/programming, /tagged/technology, /tag...",[Languages change. Languages adapt. Python 2 i...,6 New Features in Python 3.8 for Python Newbies,https://towardsdatascience.com/6-new-features-...
2,"[/tagged/python, /tagged/data-science, /tagged...","[Python is cool. Really cool. However, a lot o...",How to be fancy with Python,https://towardsdatascience.com/how-to-be-fancy...
3,"[/tagged/machine-learning, /tagged/tensorflow,...",[Convolutional neural networks (CNN) work grea...,Understanding and implementing a fully convolu...,https://towardsdatascience.com/implementing-a-...
4,"[/tagged/machine-learning, /tagged/privacy, /t...",[Data privacy has been called “the most import...,Perfectly Privacy-Preserving AI,https://towardsdatascience.com/perfectly-priva...


# Part II: Scraping Subtitles, Upvotes, Date, Name, Read Time, and Comments

Create functions for second scrape

In [6]:
def make_request2(url):
# MAKE REQUEST  
    driver = ''
    while driver == '':
        try:
            driver = webdriver.Chrome()
            driver.get(url)
            res = driver.execute_script("return document.documentElement.outerHTML")
            driver.quit()
            break
        except:
            sleep(30)
            continue
    return res

def find_post_cards(soup):
    # PULLS EACH CARD FROM THE FEED. EACH CARD IS A STORY OR COMMENT
    cards = soup.find_all("div", class_="streamItem streamItem--postPreview js-streamItem")
    return cards

def get_subtitles_from_cards(cards):
    # PULLS TITLE DATA FROM EACH CARD IN CARDS, RETURNS A LIST OF TITLES
    def subtitle_cleaner(subtitle):
        # REMOVE MEDIUMS ENCODING SYMBOLS AND EMOJIS FROM TITLES
        subtitle = subtitle.replace("\xa0"," ")
        subtitle = subtitle.replace("\u200a","")
        subtitle = subtitle.replace("\ufe0f","")
        subtitle = re.sub(r'[^\x00-\x7F]+','', subtitle)
        return subtitle

    subtitles=[]
    for card in cards:
        # SEARCH FOR TITLE THERE ARE 3 DIFF CLASSES
        variant1 = card.find("h4", class_="graf graf--h4 graf-after--h3 graf--subtitle")
        variant2 = card.find("h4", class_="graf graf--h4 graf-after--h3 graf--trailing graf--subtitle")
        variant3 = card.find("strong", class_="markup--strong markup--p-strong")
        variant4 = card.find("h4", class_="graf graf--p graf-after--h3 graf--trailing")
        variant5= card.find("p", class_="graf graf--p graf-after--h3 graf--trailing")
        variant6= card.find("blockquote", class_="graf graf--pullquote graf-after--figure graf--trailing")
        variant7 = card.find("p", class_="graf graf--p graf-after--figure")
        variant8 = card.find("blockquote", class_="graf graf--blockquote graf-after--h3 graf--trailing")
        variant9 = card.find("p", class_="graf graf--p graf-after--figure graf--trailing")
        variant10 = card.find("em", class_="markup--em markup--p-em")
        variant11=card.find("p", class_="graf graf--p graf-after--p graf--trailing")
        # EACH CARD MUST HAVE ONE OF THE TITLE CLASSES FIND IT AND CUT OUT MEDIUM'S
        # STYLING CODES
        variants = [variant1, variant2, variant3, variant4, variant5, variant6, variant7, variant8, variant9, variant10, variant11]
        saved = False
        for variant in variants:
            if ((variant is not None) and (not saved)):
                subtitle = variant.text
                subtitle = subtitle_cleaner(subtitle)
                subtitles.append(subtitle)
                saved = True
        if not saved:
            subtitles.append("NaN")
    return subtitles 

def get_urls_from_cards(cards):
    # GETS ARTICLE URLS FROM ALL CARDS
    urls = []
    for card in cards:
        url = card.find("a", class_="")
        if url is not None:
            urls.append(url['href'])
        else:
            raise Exception("couldnt find a url")
    return urls

Gather subtitle, upvotes, date, name, read time, and comments

In [7]:
# STORE RESULTS
title=[]
subtitle=[]
link=[]
upvotes=[]
date=[]
name=[]
read=[]
comments=[]

for i in months: # LOOP THROUGH MONTHS
    
    if i == '02': # FEBRUARY 2020 HAS 29 DAYS IN A MONTH
        print(i)
        for j in days29: # LOOP THROUGH DAYS

            url = 'https://towardsdatascience.com/archive/2020/'+ i + '/' + j
            res = make_request2(url)
                
            # PARSE PAGE
            soup = BeautifulSoup(res, 'lxml')
            cards = find_post_cards(soup)
            
            # SUBTITLE
            subtitle.append(get_subtitles_from_cards(cards))
            
            # URL
            link.append(get_urls_from_cards(cards))
                
            # UPVOTES
            for my_tag in soup.find_all('span',{'class':'u-relative u-background js-actionMultirecommendCount u-marginLeft5'}):
                upvotes.append(my_tag.text)
            
            # DATE
            for my_tag in soup.findAll('time'):
                if my_tag.has_attr('datetime'):
                    date.append(my_tag['datetime'])
                else:
                    date.append(my_tag.text)
                    
            # NAME
            for my_tag in soup.find_all(class_="ds-link ds-link--styleSubtle link link--darken link--accent u-accentColor--textNormal u-accentColor--textDarken"):
                name.append(my_tag.text)
             
            # READ
            for my_tag in soup.find_all(class_="readingTime"):
                read.append(my_tag.get('title'))

            # COMMENTS
            for my_tag in soup.find_all('div',{'class':'buttonSet u-floatRight'}):
                comments.append(my_tag.text)
                
            sleep(10)

    elif i == '04' or i == '06': # APRIL AND JUNE HAVE 30 DAYS IN A MONTH
        print(i)
        for j in days30: # LOOP THROUGH DAYS

            url = 'https://towardsdatascience.com/archive/2020/'+ i + '/' + j
            res = make_request2(url)
                    
            # PARSE PAGE
            soup = BeautifulSoup(res, 'lxml')
            cards = find_post_cards(soup)

            # SUBTITLE
            subtitle.append(get_subtitles_from_cards(cards))          

            # URL
            link.append(get_urls_from_cards(cards))
            
            # UPVOTES
            for my_tag in soup.find_all('span',{'class':'u-relative u-background js-actionMultirecommendCount u-marginLeft5'}):
                upvotes.append(my_tag.text)
    
            # DATE
            for my_tag in soup.findAll('time'):
                if my_tag.has_attr('datetime'):
                    date.append(my_tag['datetime'])
                else:
                    date.append(my_tag.text)
            
            # NAME
            for my_tag in soup.find_all(class_="ds-link ds-link--styleSubtle link link--darken link--accent u-accentColor--textNormal u-accentColor--textDarken"):
                name.append(my_tag.text)
                
            # READ              
            for my_tag in soup.find_all(class_="readingTime"):
                read.append(my_tag.get('title'))
 
            # COMMENTS
            for my_tag in soup.find_all('div',{'class':'buttonSet u-floatRight'}):
                comments.append(my_tag.text)
                
            sleep(10)
        
    else: # ALL OTHER MONTHS HAVE 31 DAYS 
        print(i)
        for j in days31: # LOOP THROUGH DAYS

            url = 'https://towardsdatascience.com/archive/2020/'+ i + '/' + j
            res = make_request2(url)
                    
            # PARSE PAGE
            soup = BeautifulSoup(res, 'lxml')
            cards = find_post_cards(soup)
            
            # SUBTITLE
            subtitle.append(get_subtitles_from_cards(cards))
            
            # URL
            link.append(get_urls_from_cards(cards))
                    
            # UPVOTES
            for my_tag in soup.find_all('span',{'class':'u-relative u-background js-actionMultirecommendCount u-marginLeft5'}):
                upvotes.append(my_tag.text)
    
            # DATE
            for my_tag in soup.findAll('time'):
                if my_tag.has_attr('datetime'):
                    date.append(my_tag['datetime'])
                else:
                    date.append(my_tag.text)
            
            # NAME
            for my_tag in soup.find_all(class_="ds-link ds-link--styleSubtle link link--darken link--accent u-accentColor--textNormal u-accentColor--textDarken"):
                name.append(my_tag.text)

            # READ
            for my_tag in soup.find_all(class_="readingTime"):
                read.append(my_tag.get('title'))
                
            # COMMENTS
            for my_tag in soup.find_all('div',{'class':'buttonSet u-floatRight'}):
                comments.append(my_tag.text)
                
            sleep(10)

01
02
03
04
05
06
07


Flatten title, subtitle, and url lists

In [8]:
flattened_subtitle = [y for x in subtitle for y in x]
flattened_link = [y for x in link for y in x]

Convert to dataframe 

In [9]:
df2 = pd.DataFrame({'Subtitle':flattened_subtitle,'Url':flattened_link,'Name':name,'Upvotes':upvotes,'Date':date,'Comments':comments, "Read": read})
df2.head()

,Subtitle,Url,Name,Upvotes,Date,Comments,Read
0,Lets look at the performance of our Python pro...,https://towardsdatascience.com/making-python-p...,Martin Heinz,3.3K,2020-01-01T20:15:03.352Z,4 responses,5 min read
1,Prepare yourself as,https://towardsdatascience.com/6-new-features-...,Eden Au,1.7K,2020-01-01T23:27:59.289Z,4 responses,4 min read
2,Python tricks that will make your life easier,https://towardsdatascience.com/how-to-be-fancy...,Dipam Vasani,1.7K,2020-01-01T14:26:52.211Z,12 responses,5 min read
3,"A tutorial on building, training and",https://towardsdatascience.com/implementing-a-...,Himanshu Rawlani,271,2020-01-01T16:01:59.680Z,1 response,10 min read
4,What is it and how do we achieve it?,https://towardsdatascience.com/perfectly-priva...,Patricia Thaine,206,2020-01-01T23:46:51.168Z,1 response,10 min read


# Part III: Combine Dataframes

Now merge both dataframes

In [11]:
# First remove '-----' from URL to make URLs same across both dataframes
df1['Url'] = df1['Url'].str.split('-----').str[0]
df2['Url'] = df2['Url'].str.split('-----').str[0]

# Merge
df_m = pd.merge(left=df1, right=df2, left_on='Url', right_on='Url')
df_m.head()

,Tags,Text,Title,Url,Subtitle,Name,Upvotes,Date,Comments,Read
0,"[/tagged/programming, /tagged/python, /tagged/...","[Python haters always say, that one of the rea...",Making Python Programs Blazingly Fast,https://towardsdatascience.com/making-python-p...,Lets look at the performance of our Python pro...,Martin Heinz,3.3K,2020-01-01T20:15:03.352Z,4 responses,5 min read
1,"[/tagged/programming, /tagged/technology, /tag...",[Languages change. Languages adapt. Python 2 i...,6 New Features in Python 3.8 for Python Newbies,https://towardsdatascience.com/6-new-features-...,Prepare yourself as,Eden Au,1.7K,2020-01-01T23:27:59.289Z,4 responses,4 min read
2,"[/tagged/python, /tagged/data-science, /tagged...","[Python is cool. Really cool. However, a lot o...",How to be fancy with Python,https://towardsdatascience.com/how-to-be-fancy...,Python tricks that will make your life easier,Dipam Vasani,1.7K,2020-01-01T14:26:52.211Z,12 responses,5 min read
3,"[/tagged/machine-learning, /tagged/tensorflow,...",[Convolutional neural networks (CNN) work grea...,Understanding and implementing a fully convolu...,https://towardsdatascience.com/implementing-a-...,"A tutorial on building, training and",Himanshu Rawlani,271,2020-01-01T16:01:59.680Z,1 response,10 min read
4,"[/tagged/machine-learning, /tagged/privacy, /t...",[Data privacy has been called “the most import...,Perfectly Privacy-Preserving AI,https://towardsdatascience.com/perfectly-priva...,What is it and how do we achieve it?,Patricia Thaine,206,2020-01-01T23:46:51.168Z,1 response,10 min read


In [13]:
path = "/Users/halabanz/Desktop/tds_analysis/df.csv"
df_m.to_csv(path, index = None, header = True)